In [1]:
import agate

# This is how we set up the data to show changes between 95/96 and 15/16

In [2]:
# bring data in
specified_types = {
    'CAMPUS': agate.Text()
}
#demographic data
students9596 = agate.Table.from_csv('../raw_data/stud9596.csv')
students1516 = agate.Table.from_csv('../raw_data/1516student.txt')
#campus data
cr9596 = agate.Table.from_csv('../raw_data/campref.csv')
cr1516 = agate.Table.from_csv('../raw_data/1516cr.txt')
locations = agate.Table.from_csv('../raw_data/Current_Schools.csv')


### Join the student and campus data together. 

In [3]:
#join student and campus data
join9596 = students9596.join(cr9596, 'CAMPUS')
join1516 = students1516.join(cr1516, 'CAMPUS')

#check that everything made it
print(join9596)

| column   | data_type |
| -------- | --------- |
| CAMPUS   | Number    |
| CPETECOP | Number    |
| CPETECOC | Number    |
| CPETBILC | Number    |
| CPETALLC | Number    |
| CPETINDC | Number    |
| CPETPACC | Number    |
| CPETBLAC | Number    |
| CPETHISC | Number    |
| CPETWHIC | Number    |
| CPETWHIP | Number    |
| CPETBLAP | Number    |
| CPETHISP | Number    |
| CPETINDP | Number    |
| CPETPACP | Number    |
| CPETBILP | Number    |
| DISTNAME | Text      |
| CAMPNAME | Text      |
| CLASS    | Text      |



## Select needed columns, then rename them

In [4]:
#sets of columns, then selects them
columns_one = ['CAMPUS', 'CPETALLC','CPETBLAC', 'CPETHISC', 'CPETWHIC', 'CPETINDC', 'CPETPACC',
        'CPETBLAP','CPETHISP','CPETWHIP', 'CPETINDP', 'CPETPACP','CPETBILC', 'DISTNAME', 'CAMPNAME']

columns_three = ['CAMPUS', 'CPETALLC','CPETBLAC', 'CPETHISC', 'CPETWHIC', 'CPETINDC', 'CPETASIC', 'CPETPCIC', 'CPETTWOC',
       'CPETECOC', 'CPETLEPC','CPETBLAP','CPETHISP','CPETWHIP','CPETINDP', 'CPETASIP', 'CPETPCIP', 'CPETTWOP', 'CPETECOP','CPETLEPP', 'DISTNAME', 'CAMPNAME']
 
campus9596_select = join9596.select(columns_one)
campus1516_select = join1516.select(columns_three)

## Rename all of the columns - the years are very helpful if/when the date column doesn't work/transfer

In [5]:
#rename
column_names_one = {'CAMPUS': 'CAMPUS',
                                       'CPETALLC': 'ALL_9596',
                                       'CPETBLAC': 'B_9596',
                                       'CPETHISC': 'H_9596',
                                       'CPETWHIC': 'W_9596',
                                       'CPETINDC': 'AI_9596',
                                       'CPETPACC': 'API_9596',
                                       'CPETBLAP': 'BP_9596',
                                       'CPETHISP': 'HP_9596',
                                       'CPETWHIP': 'WP_9596',
                                       'CPETINDP': 'AIP_9596',
                                       'CPETPACP': 'APIP_9596',
                                       'CPETBILC': 'BILINGUAL_ENROLL'}
column_names_three = {'CAMPUS': 'CAMPUS',
                                       'CPETALLC': 'ALL_1516',
                                       'CPETBLAC': 'B_1516',
                                       'CPETHISC': 'H_1516',
                                       'CPETWHIC': 'W_1516',
                                       'CPETINDC': 'AI_1516',
                                       'CPETASIC': 'AS_1516',
                                       'CPETPCIC': 'PI_1516',
                                       'CPETTWOC': 'TWO_1516',
                                       'CPETECOC': 'EC_1516',
                                       'CPETLEPC': 'EL_1516',
                                       'CPETBLAP': 'BP_1516',
                                       'CPETHISP': 'HP_1516',
                                       'CPETWHIP': 'WP_1516',
                                       'CPETINDP': 'AIP_1516',
                                       'CPETASIP': 'ASP_1516',
                                       'CPETPCIP': 'PIP_1516',
                                       'CPETTWOP': 'TWOP_1516',                
                                       'CPETECOP': 'ECP_1516',
                                       'CPETLEPP': 'ELP_1516',
                                       'CPETBILC': 'BILINGUAL_ENROLL'}
campus9596_rename = campus9596_select.rename(column_names_one)
campus1516_rename = campus1516_select.rename(column_names_three)

## Set up the date column - useful for line graphs and telling where the data came from

In [6]:
#set up date column
class datecolumn(agate.Computation): 
    def get_computed_data_type(self, table):
        return agate.Text()
    def run(self, table):
        new_column = []
        for row in table.rows:
            new_column.append(year_value)
        return new_column
#add 0607 date column    
year_value = '1996-01-01'
campus9596_date = campus9596_rename.compute([
    ('year', datecolumn())
])

In [7]:
#repeat for 1516
year_value = '2016-01-01'
campus1516_date = campus1516_rename.compute([
    ('year', datecolumn())
])

In [8]:
#join them together
combo = campus1516_date.join(campus9596_date, 'CAMPUS')

In [9]:
## Now filter to needed districts.

district_ids = ['AUSTIN ISD', 'PFLUGERVILLE ISD', 'MANOR ISD']

filteredcombo = combo.where(lambda row: row['DISTNAME'].upper() in district_ids
                              )
## We originally had a set of 15 districts in the area, but ultimately used these three per Melissa. A full district
## list can be pulled from the campus_full notebook. 


## Join locations and campus tables so we can map the data

In [10]:
#add geo data
located = filteredcombo.join(locations, 'CAMPUS')

/Users/okrauth/anaconda3/envs/data/lib/python3.6/site-packages/agate/utils.py:291: DuplicateColumnWarning: Column name "DISTNAME2" already exists in Table. Column will be renamed to "DISTNAME2_2".
/Users/okrauth/anaconda3/envs/data/lib/python3.6/site-packages/agate/utils.py:291: DuplicateColumnWarning: Column name "CAMPNAME2" already exists in Table. Column will be renamed to "CAMPNAME2_2".


In [11]:
latlong = {'\ufeffX': 'longitude',
          'Y': 'latitude'}
located = located.rename(latlong)
#fix funky name for longitude - this has been fixing the problem, but if you get an error, check this first. 

## Filter the columns and apply

In [12]:
#select columns
location_columns = ['DISTNAME', 'CAMPNAME', 'longitude', 'latitude','STREET', 'CITY', 'ZIP','CAMPUS','year',
                    'ALL_9596','ALL_1516','B_9596','B_1516','H_9596','H_1516','W_9596','W_1516','AI_9596','AI_1516',
                    'API_9596','AS_1516','PI_1516','TWO_1516','EC_1516','EL_1516',
                    'BP_9596','BP_1516','HP_9596','HP_1516','WP_9596','WP_1516','AIP_9596','AIP_1516',
                    'APIP_9596','ASP_1516','PIP_1516','TWOP_1516','ECP_1516','ELP_1516']

In [13]:
#select columns pt 2
locationfiltertwo = located.select(location_columns)
locationfiltertwo.to_csv(path='locationfiltertwo.csv')

## Add Asian and Pacific Islander columns together in 15/16

In [14]:
#This allows the data to stay consistent between both years since A/PI became separate columns around 2011/12. 

def f(row):
    return (row['AS_1516'] + row['PI_1516'])

asiancombo = locationfiltertwo.compute([
    ('API_1516', agate.Formula(agate.Number(), f))
])

def f(row):
    return (row['ASP_1516'] + row['PIP_1516'])

fullcombo = asiancombo.compute([
    ('APIP_1516', agate.Formula(agate.Number(), f))
])



## Create change columns to see the difference between the two years

In [15]:
#This will create new columns to show the change.
changecombo = fullcombo.compute([
    ('ALL_CHANGE', agate.Change('ALL_9596', 'ALL_1516')),
    ('B_CHANGE', agate.Change('B_9596', 'B_1516')),
    ('H_CHANGE', agate.Change('H_9596', 'H_1516')),
    ('W_CHANGE', agate.Change('W_9596', 'W_1516')),
    ('AI_CHANGE', agate.Change('AI_9596', 'AI_1516')),
    ('API_CHANGE', agate.Change('API_9596', 'API_1516')),
    ('BP_CHANGE', agate.Change('BP_9596', 'BP_1516')),
    ('HP_CHANGE', agate.Change('HP_9596', 'HP_1516')),
    ('WP_CHANGE', agate.Change('WP_9596', 'WP_1516')),
    ('AIP_CHANGE', agate.Change('AIP_9596', 'AIP_1516')),
    ('APIP_CHANGE', agate.Change('APIP_9596', 'APIP_1516'))
  ])

/Users/okrauth/anaconda3/envs/data/lib/python3.6/site-packages/agate/computations/change.py:49: NullCalculationWarning: Column "ALL_9596" contains nulls. These will be excluded from Change calculation.
/Users/okrauth/anaconda3/envs/data/lib/python3.6/site-packages/agate/computations/change.py:49: NullCalculationWarning: Column "B_9596" contains nulls. These will be excluded from Change calculation.
/Users/okrauth/anaconda3/envs/data/lib/python3.6/site-packages/agate/computations/change.py:49: NullCalculationWarning: Column "H_9596" contains nulls. These will be excluded from Change calculation.
/Users/okrauth/anaconda3/envs/data/lib/python3.6/site-packages/agate/computations/change.py:49: NullCalculationWarning: Column "W_9596" contains nulls. These will be excluded from Change calculation.
/Users/okrauth/anaconda3/envs/data/lib/python3.6/site-packages/agate/computations/change.py:49: NullCalculationWarning: Column "AI_9596" contains nulls. These will be excluded from Change calculatio

### Send to CSV - we use this and the next CSV to get a full range of information in Tableau

In [16]:
changecombo.to_csv(path='changecombo9516.csv')

## Make columns with race/95 number/15 number/change for both the counts and the percentages

In [17]:
#making some combos
combos = changecombo.compute([
    ('B_combo', agate.Formula(agate.Text(), lambda row: 'BLACK'\
                              + "|"
                              + str(row['B_9596'])\
                              + "|"
                              + str(row['B_1516'])\
                              + "|"
                              + str(row['B_CHANGE'])\
                             )),
    ('H_combo', agate.Formula(agate.Text(), lambda row: 'HISPANIC'\
                              + "|"
                              + str(row['H_9596'])\
                              + "|"
                              + str(row['H_1516'])\
                              + "|"
                              + str(row['H_CHANGE'])\
                             )),
    ('W_combo', agate.Formula(agate.Text(), lambda row: 'WHITE'\
                              + "|"
                              + str(row['W_9596'])\
                              + "|"
                              + str(row['W_1516'])\
                              + "|"
                              + str(row['W_CHANGE'])\
                             )),
    ('AI_combo', agate.Formula(agate.Text(), lambda row: 'INDIAN'\
                              + "|"
                              + str(row['AI_9596'])\
                              + "|"
                              + str(row['AI_1516'])\
                              + "|"
                              + str(row['AI_CHANGE'])\
                             )),
    ('API_combo', agate.Formula(agate.Text(), lambda row: 'API'\
                              + "|"
                              + str(row['API_9596'])\
                              + "|"
                              + str(row['API_1516'])\
                              + "|"
                              + str(row['API_CHANGE'])\
                             )),
    ('BP_combo', agate.Formula(agate.Text(), lambda row: 'BLACKP'\
                              + "|"
                              + str(row['BP_9596'])\
                              + "|"
                              + str(row['BP_1516'])\
                              + "|"
                              + str(row['BP_CHANGE'])\
                             )),
    ('HP_combo', agate.Formula(agate.Text(), lambda row: 'HISPANICP'\
                              + "|"
                              + str(row['HP_9596'])\
                              + "|"
                              + str(row['HP_1516'])\
                              + "|"
                              + str(row['HP_CHANGE'])\
                             )),
    ('WP_combo', agate.Formula(agate.Text(), lambda row: 'WHITEP'\
                              + "|"
                              + str(row['WP_9596'])\
                              + "|"
                              + str(row['WP_1516'])\
                              + "|"
                              + str(row['WP_CHANGE'])\
                             )),
    ('AIP_combo', agate.Formula(agate.Text(), lambda row: 'INDIANP'\
                              + "|"
                              + str(row['AIP_9596'])\
                              + "|"
                              + str(row['AIP_1516'])\
                              + "|"
                              + str(row['AIP_CHANGE'])\
                             )),
    ('APIP_combo', agate.Formula(agate.Text(), lambda row: 'APIP'\
                              + "|"
                              + str(row['APIP_9596'])\
                              + "|"
                              + str(row['APIP_1516'])\
                              + "|"
                              + str(row['APIP_CHANGE'])\
                             )),
])

#make sure it works
print(combos)

| column      | data_type |
| ----------- | --------- |
| DISTNAME    | Text      |
| CAMPNAME    | Text      |
| longitude   | Number    |
| latitude    | Number    |
| STREET      | Text      |
| CITY        | Text      |
| ZIP         | Number    |
| CAMPUS      | Number    |
| year        | Text      |
| ALL_9596    | Number    |
| ALL_1516    | Number    |
| B_9596      | Number    |
| B_1516      | Number    |
| H_9596      | Number    |
| H_1516      | Number    |
| W_9596      | Number    |
| W_1516      | Number    |
| AI_9596     | Number    |
| AI_1516     | Number    |
| API_9596    | Number    |
| AS_1516     | Number    |
| PI_1516     | Number    |
| TWO_1516    | Number    |
| EC_1516     | Number    |
| EL_1516     | Number    |
| BP_9596     | Number    |
| BP_1516     | Number    |
| HP_9596     | Number    |
| HP_1516     | Number    |
| WP_9596     | Number    |
| WP_1516     | Number    |
| AIP_9596    | Number    |
| AIP_1516    | Number    |
| APIP_9596   | Numb

#### Select columns and create a new CSV

In [18]:
#focus on school and combo columns
school_columns = (
    'DISTNAME',
    'CAMPNAME',
    'longitude',
    'latitude',
    'STREET',
    'CITY',
    'ZIP',
    'CAMPUS',
    'year',
    'ALL_9596',
    'ALL_1516'
)

combo_columns = ('B_combo', 'H_combo', 'W_combo', 'AI_combo','API_combo','BP_combo','HP_combo',
                 'WP_combo', 'AIP_combo', 'APIP_combo')
campuscombocol = school_columns + combo_columns
campuscombo = combos.select(campuscombocol)
campuscombo.to_csv(path='campuscombo.csv')

### Normalize 

In [19]:
data_normalized = campuscombo.normalize(
        list(school_columns),
        list(combo_columns)
    )
print(data_normalized) # check columns
data_normalized.select(['CAMPNAME', 'property', 'value']).print_table() # peek at important cols

| column    | data_type |
| --------- | --------- |
| DISTNAME  | Text      |
| CAMPNAME  | Text      |
| longitude | Number    |
| latitude  | Number    |
| STREET    | Text      |
| CITY      | Text      |
| ZIP       | Number    |
| CAMPUS    | Number    |
| year      | Text      |
| ALL_9596  | Number    |
| ALL_1516  | Number    |
| property  | Text      |
| value     | Text      |

| CAMPNAME   | property   | value                |
| ---------- | ---------- | -------------------- |
| AUSTIN H S | B_combo    | BLACK|171|100|-71    |
| AUSTIN H S | H_combo    | HISPANIC|829|919|90  |
| AUSTIN H S | W_combo    | WHITE|1215|956|-259  |
| AUSTIN H S | AI_combo   | INDIAN|5|2|-3        |
| AUSTIN H S | API_combo  | API|35|42|7          |
| AUSTIN H S | BP_combo   | BLACKP|7.6|4.8|-2.8  |
| AUSTIN H S | HP_combo   | HISPANICP|36.8|44... |
| AUSTIN H S | WP_combo   | WHITEP|53.9|46|-7.9  |
| AUSTIN H S | AIP_combo  | INDIANP|0.2|0.1|-0.1 |
| AUSTIN H S | APIP_combo | APIP|1.6|2.0|0.4    

### Now split it to have the yearly and change data in separate columns

In [20]:
# split on the pipes, will get the category, the 95 amount, the 15 amount, and the change between the two. 
exploded = data_normalized.compute([
        ('CATEGORY', agate.Formula(agate.Text(), lambda row: row['value'].split("|")[0])),
        ('9596', agate.Formula(agate.Text(), lambda row: row['value'].split("|")[1])),
        ('1516', agate.Formula(agate.Text(), lambda row: row['value'].split("|")[2])),
        ('CHANGE', agate.Formula(agate.Text(), lambda row: row['value'].split("|")[3])),
    ])
print(exploded)
exploded.select(['CAMPNAME', 'value', 'CATEGORY', '9596', '1516']).print_table() # just to peek at it

| column    | data_type |
| --------- | --------- |
| DISTNAME  | Text      |
| CAMPNAME  | Text      |
| longitude | Number    |
| latitude  | Number    |
| STREET    | Text      |
| CITY      | Text      |
| ZIP       | Number    |
| CAMPUS    | Number    |
| year      | Text      |
| ALL_9596  | Number    |
| ALL_1516  | Number    |
| property  | Text      |
| value     | Text      |
| CATEGORY  | Text      |
| 9596      | Text      |
| 1516      | Text      |
| CHANGE    | Text      |

| CAMPNAME   | value                | CATEGORY  | 9596 | 1516 |
| ---------- | -------------------- | --------- | ---- | ---- |
| AUSTIN H S | BLACK|171|100|-71    | BLACK     | 171  | 100  |
| AUSTIN H S | HISPANIC|829|919|90  | HISPANIC  | 829  | 919  |
| AUSTIN H S | WHITE|1215|956|-259  | WHITE     | 1215 | 956  |
| AUSTIN H S | INDIAN|5|2|-3        | INDIAN    | 5    | 2    |
| AUSTIN H S | API|35|42|7          | API       | 35   | 42   |
| AUSTIN H S | BLACKP|7.6|4.8|-2.8  | BLACKP    | 7.6  | 

In [21]:
# move to a CSV 
## Tableau note - to get all of the information you need to make the current map, you need to use both CSVs you make in
## this notebook. Most of the numbers from exploded.csv are used to make the tooltip table. 

exploded.to_csv(path='exploded.csv')